# Weed & Rose Detection with Ultralytics YOLO11n
This notebook trains a YOLO11n model to detect weeds and roses from an annotated dataset of segmentation masks. It defines environment settings, configuration parameters, and the training process — with options for augmentation, reproducibility, and automatic folder management.

In [ ]:
# Make sure the notebook kernel is the conda env: Python (yolo)
import sys, platform, torch, os
import albumentations as A
print("Python:", sys.version)
print("OS:", platform.platform())
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("Albumentations version:", A.__version__)

In [ ]:
from pathlib import Path
import yaml

PROJECT_ROOT = Path(r"D:/Ai Systems Group")
DATA_YAML    = PROJECT_ROOT / "data/weeds_yolo_mask/data.yaml"
OUT_DIR      = PROJECT_ROOT / "work_dirs"

CFG = {
    "data": str(DATA_YAML),
    "model": "yolo11n-seg.pt",
    "epochs": 150,
    "batch": 9,
    "imgsz": 1024,
    "workers": 6,
    "device": 0,
    "project": str(OUT_DIR),
    "name": "yolov11n-seg-weeds",
    "exist_ok": False,
    "plots": True,
    "patience": 50,
    "seed": 42,
    # "close_mosaic": 10,
    # "cache": "ram",
}


In [ ]:
with open(DATA_YAML, "r", encoding="utf-8") as f:
    ds = yaml.safe_load(f)
print(ds)


# 3. Albumentations Augmentation Pipeline

This cell defines the **Albumentations-based image augmentations** that complement YOLO’s built-in transformations.
Albumentations provides additional *pixel-level augmentations* that help the model generalize better to real-world variations in lighting, texture, and camera quality.

---

### What it does
1. **Imports the Albumentations library** and defines a custom `Compose` pipeline named `albu_train`.
2. **Applies lightweight, complementary augmentations** on top of YOLO’s geometric ones:
   - `MotionBlur` and `MedianBlur` simulate camera shake and focus variation.
   - `RandomBrightnessContrast` adjusts brightness and contrast to improve robustness to lighting changes.
   - `CLAHE` (Contrast Limited Adaptive Histogram Equalization) enhances local contrast and texture details.
   - `ToGray` occasionally converts images to grayscale so the model doesn’t over-rely on color cues.
   - `ImageCompression` reduces quality to mimic low-end cameras or image uploads.
   - `GaussNoise` adds small random sensor noise, improving tolerance to imperfect captures.
3. Each transformation has a **low probability (`p`)** to prevent over-augmentation while maintaining dataset diversity.
4. When the `albumentations` package is installed, YOLOv8 automatically detects and integrates these augmentations during training, enhancing the overall robustness of your weed-vs-rose detection model.

In [9]:
import albumentations as A
import math

albu_train = A.Compose([
    A.MotionBlur(p=0.02, blur_limit=5),
    A.MedianBlur(p=0.02, blur_limit=5),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
    A.CLAHE(clip_limit=(1.0, 3.0), tile_grid_size=(8, 8), p=0.3),
    A.ToGray(p=0.05),
    A.ImageCompression(quality_range=(70, 100), p=0.2),

    A.GaussNoise(
        std_range=(math.sqrt(10.0) / 255.0, math.sqrt(40.0) / 255.0),
        mean_range=(0, 0),
        p=0.15
    ),
])

# 4. Model Training
This is the main training step of the notebook.
It loads the YOLOv8n model, applies all the training configurations defined earlier (`CFG`), and begins the training process using the specified dataset and augmentations.

---

### What it does
1. **Imports and loads the YOLO model:**
   - `model = YOLO(CFG["model"])` loads the base model architecture (`yolov8n.pt`), which is the lightweight **nano version** of YOLOv8, optimized for speed and smaller datasets.
2. **Trains the model using** `model.train(...)` with all your specified parameters:
   - Dataset path (`data`)
   - Number of epochs
   - Image size and batch size
   - GPU device index
   - Output directory and run name
   - Early stopping settings (`patience`)
   - Random seed for reproducibility
3. **Includes multiple image augmentations** (both built-in YOLO and Albumentations-level options) to improve generalization.
4. **Saves results automatically** in your `OUT_DIR` folder under an experiment name such as:

In [ ]:
from ultralytics import YOLO

model = YOLO(CFG["model"])
results = model.train(
    data=CFG["data"],
    epochs=CFG["epochs"],
    imgsz=CFG["imgsz"],
    batch=CFG["batch"],
    workers=CFG["workers"],
    device=CFG["device"],
    project=CFG["project"],
    name=CFG["name"],
    exist_ok=CFG["exist_ok"],
    patience=CFG["patience"],
    seed=CFG["seed"],
    plots=True,
    cls=1.0,
    weight_decay = 0.001,
    lrf = 0.0005,

    # --Augmentations--
    hsv_h=0.015, hsv_s=0.50, hsv_v=0.35,
    degrees=15.0, translate=0.06, scale=0.20, shear=5.0,
    perspective=0.0,
    flipud=0.0, fliplr=0.5,
    mosaic=0.5,
    mixup=0.1,
    close_mosaic=10,
    augmentations=albu_train
)
results


# View training curves

In [ ]:
from IPython.display import display
from PIL import Image

run_dir = OUT_DIR / "yolov11n-weeds"
img = run_dir / "results.png"
if img.exists():
    display(Image.open(img))
else:
    print("results.png not found yet:", img)


# Validate on val set (metrics + plots)

In [ ]:
metrics = model.val(
    data=CFG["data"],
    project=str(OUT_DIR),
    name=f"{CFG['name']}-val",
    plots=True
)
metrics


# Predict on some images

In [ ]:
VAL_IMAGES = Path(ds["path"]) / ds["val"]  # images/val folder
pred = model.predict(
    source=str(VAL_IMAGES),
    project=str(OUT_DIR),
    name=f"{CFG['name']}-pred",
    save=True,      # save images with drawn boxes
    save_txt=True,  # save YOLO txt predictions
    conf=0.4       # tweak for precision/recall trade-off
)
pred[:2]


# Visualize predictions
If you want to visualize the predictions that the model did, you can find them in the `work_dir` folder, in the folder that has the extension `-pred` at the end of the name.